In [2]:
!pip install tensorflow==1.15

     -------------------------------------- 295.1/295.1 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 11.6 MB/s eta 0:00:00
     --------------------------------------- 14.0/14.0 MB 11.5 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 11.4 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ------------------------------------- 525.0/525.0 kB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB ? eta 0:00:00
     ------------------------------------- 503.4/503.4 kB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 50.7/50.7 kB ? eta 0:00:00
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
     ---------------------------------------- 2.6/2.6 MB 11.1 MB/s eta 0:

In [1]:
import tensorflow as tf

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\leysh\anaconda3\envs\pavlov\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\leysh\AppData\Local\Temp\ipykernel_26360\3793406994.py", line 1, in <module>
    import tensorflow as tf
  File "c:\Users\leysh\anaconda3\envs\pavlov\lib\site-packages\tensorflow\__init__.py", line 99, in <module>
    from tensorflow_core import *
  File "c:\Users\leysh\anaconda3\envs\pavlov\lib\site-packages\tensorflow_core\__init__.py", line 28, in <module>
    from tensorflow.python import pywrap_tensorflow  # pylint: disable=unused-import
  File "<frozen importlib._bootstrap>", line 1019, in _handle_fromlist
  File "c:\Users\leysh\anaconda3\envs\pavlov\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "c:\Users\leysh\anaconda3\envs\pavlov\lib\site-packages\tensorflow\__init__.py", line 44, in _load
    module 

In [26]:
tf.__version__

'1.15.0'

In [27]:
!pip install deeppavlov

In [28]:
ROOT_PATH = "deeppavlov1"
DOWNLOADS_PATH = ROOT_PATH + "/downloads"
MODELS_PATH = ROOT_PATH + "/models"

In [30]:
!ls

 content      deeppavlov1   MuSeRC	 'Seminar 6'
 DaNetQA      logs	    paraphraser   Untitled0.ipynb
 deeppavlov   muserc	    res.txt	 'Без названия1.ipynb'


In [31]:
from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader

reader_pp = ParaphraserReader()
data_pp = reader_pp.read('paraphraser', do_lower_case=False)

In [32]:
data_pp['test'][0]

(('Цены на нефть восстанавливаются',
  'Парламент Словакии поблагодарил народы бывшего СССР за Победу'),
 0)

In [33]:
from deeppavlov.dataset_iterators.siamese_iterator import SiameseIterator

iterator_pp = SiameseIterator(data=data_pp, seed=243, len_valid=500)

In [40]:
chainer_config_pp = dict()
chainer_config_pp['in'] = ['text_a', 'text_b']
chainer_config_pp['in_y'] = ['y']
preproc_pp = {'class_name': "bert_preprocessor", 
            'vocab_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
           'do_lower_case': False,
           'max_seq_length': 64,
           'in': ['text_a', 'text_b'],
           'out': ['bert_features']}
classifier_pp = {'class_name': 'bert_classifier',
              'n_classes': 2,
              'one_hot_labels': False,
              'bert_config_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
              'pretrained_bert': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
              'save_path': "paraphraser/paraph_only/paraph_colab",
              'load_path': "paraphraser/paraph_only/paraph",
              'keep_prob': 0.5,
              'optimizer': 'tf.train.AdamOptimizer',
              'learning_rate': 2e-5,
              'learning_rate_drop_patience': 3,
              'learning_rate_drop_div': 2,
              'in': ['bert_features'],
              'in_y': ['y'],
              'out': ['predictions']}
chainer_config_pp['pipe'] = [preproc_pp, classifier_pp]
chainer_config_pp['out'] = ['predictions']

In [43]:
from deeppavlov.core.trainers import NNTrainer

trainer_pp = NNTrainer(chainer_config=chainer_config_pp, batch_size=64,
                       train_metrics=['acc'], metrics=['acc'],
                       validation_patience=7, val_every_n_batches=50, log_every_n_batches=50,
                       val_every_n_epochs=1, log_every_n_epochs=1,
                       tensorboard_log_dir=MODELS_PATH + '/paraphraser_rubert/logs')

In [44]:
trainer_pp.train(iterator_pp)

2021-05-13 13:35:13.692 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/paraphraser/paraph_only/paraph]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/paraphraser/paraph_only/paraph


2021-05-13 13:38:22.914 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best acc of 0.888


{"valid": {"eval_examples_count": 500, "metrics": {"acc": 0.888}, "time_spent": "0:02:55", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 7}}
{"train": {"eval_examples_count": 64, "metrics": {"acc": 1.0}, "time_spent": "0:42:21", "epochs_done": 0, "batches_seen": 50, "train_examples_seen": 3200, "learning_rate": 2e-05, "loss": 0.047718408117070794}}


2021-05-13 14:21:07.932 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.888


...

2021-05-13 16:34:32.7 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.89
2021-05-13 16:34:32.712 INFO in 'deeppavlov.core.models.lr_scheduled_model'['lr_scheduled_model'] at line 429: New learning rate dividor = 2.0


{"valid": {"eval_examples_count": 500, "metrics": {"acc": 0.876}, "time_spent": "2:59:04", "epochs_done": 1, "batches_seen": 200, "train_examples_seen": 12782, "impatience": 3, "patience_limit": 7}}
{"train": {"eval_examples_count": 64, "metrics": {"acc": 1.0}, "time_spent": "3:07:05", "epochs_done": 2, "batches_seen": 210, "train_examples_seen": 13404, "learning_rate": 1e-05, "loss": 0.03871533186174929}}


2021-05-13 16:44:35.742 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.89


{"valid": {"eval_examples_count": 500, "metrics": {"acc": 0.878}, "time_spent": "3:09:08", "epochs_done": 2, "batches_seen": 210, "train_examples_seen": 13404, "impatience": 4, "patience_limit": 7}}


2021-05-13 17:08:17.483 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 339: Stopped training


In [ ]:
!pip install deeppavlov

In [ ]:
from deeppavlov.dataset_readers.boolqa_reader import BoolqaReader

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd DaNetQA

/content/drive/My Drive/colab/DaNetQA


In [ ]:
import json

In [ ]:
def read(data_path, language, *args, **kwargs):
    dataset = {}

    for filename in ['train.jsonl', 'valid.jsonl', 'test.jsonl']:
        dataset[filename.split('.')[0]] = build_data(language, data_path + filename)

    return dataset

def build_data(ln, data_path):

    data = {}
    with open(data_path, 'r') as f:
        for line in f:
            jline = json.loads(line)
            if ln == 'ru':
                if 'label' in jline:
                    data[jline['question'], jline['passage']] = int(jline['label'])
                else:
                  data[jline['question']] = jline['passage']

    return list(data.items())

In [ ]:
data['test'][0]

('Полезна ли ртуть с градусника?',
 'Отравления ртутью  — расстройства здоровья, связанные с избыточным поступлением паров или соединений ртути в организм. Токсические свойства ртути известны с глубокой древности. Соединения ртути — киноварь, каломель и сулема — применялись для разных целей, в том числе и в качестве ядов. С древних времён известна также и металлическая ртуть, хотя её токсичность поначалу сильно недооценивалась. Ртуть и её соединения стали особенно широко применяться в средние века, в частности при производстве золота и серебряных зеркал, а также при изготовлении фетра для шляп, что вызвало поток новых, уже профессиональных отравлений.')

In [ ]:
data = read('', 'ru')

In [ ]:
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

iterator = BasicClassificationDatasetIterator(data=data, seed=243)

In [ ]:
chainer_config = dict()
chainer_config['in'] = ['text_a', 'text_b']
chainer_config['in_y'] = ['y']
preproc = {'class_name': "bert_preprocessor", 
            'vocab_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
           'do_lower_case': False,
           'max_seq_length': 128,
           'in': ['text_a', 'text_b'],
           'out': ['bert_features']}
classifier = {'class_name': 'bert_classifier',
              'n_classes': 2,
              'one_hot_labels': False,
              'bert_config_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
              'pretrained_bert': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
              'save_path': 'muserc/danetqa_muserc',
              'load_path': 'muserc/danetqa_muserc', #
              'keep_prob': 0.5,
              'optimizer': 'tf.train.AdamOptimizer',
              'learning_rate': 2e-5, #
              'learning_rate_decay': 'linear',
              'learning_rate_decay_epochs': 40, #
              'in': ['bert_features'],
              'in_y': ['y'],
              'out': ['predictions']}
chainer_config['pipe'] = [preproc, classifier]
chainer_config['out'] = ['predictions']

In [ ]:
from deeppavlov.core.trainers import NNTrainer

In [ ]:
trainer = NNTrainer(chainer_config=chainer_config, epochs=40, batch_size=32,
                       train_metrics=['acc'], metrics=['acc'], #
                       val_every_n_epochs=1, log_every_n_epochs=1,
                       evaluation_targets=['valid', 'train'], tensorboard_log_dir=MODELS_PATH + '/boolqa_rubert/logs')

In [ ]:
%cd ..

/content/drive/My Drive/colab


In [ ]:
trainer.train(iterator)

2021-05-09 14:14:30.932 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/muserc/danetqa_muserc]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/muserc/danetqa_muserc


2021-05-09 14:22:46.506 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best acc of 0.6264


{"valid": {"eval_examples_count": 819, "metrics": {"acc": 0.6264}, "time_spent": "0:08:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 1.0}, "time_spent": "0:53:11", "epochs_done": 1, "batches_seen": 55, "train_examples_seen": 1749, "learning_rate": 2e-05, "loss": 0.040479276392778214}}


2021-05-09 15:16:49.550 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.6264


...

2021-05-09 17:58:03.113 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.6264


{"valid": {"eval_examples_count": 819, "metrics": {"acc": 0.6129}, "time_spent": "3:43:18", "epochs_done": 4, "batches_seen": 220, "train_examples_seen": 6996, "impatience": 4, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 1.0}, "time_spent": "4:28:41", "epochs_done": 5, "batches_seen": 275, "train_examples_seen": 8745, "learning_rate": 1.8e-05, "loss": 0.0027249261175959624}}


2021-05-09 18:52:17.3 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.6264


{"valid": {"eval_examples_count": 819, "metrics": {"acc": 0.6056}, "time_spent": "4:37:32", "epochs_done": 5, "batches_seen": 275, "train_examples_seen": 8745, "impatience": 5, "patience_limit": 5}}


2021-05-09 18:52:18.240 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 329: Ran out of patience


In [ ]:
data_pp['train'][0]

(('Полицейским разрешат стрелять на поражение по гражданам с травматикой.',
  'Полиции могут разрешить стрелять по хулиганам с травматикой.'),
 1)

In [ ]:
class TERRaReader:
    def read(self, data_path: str):
        dataset = {}
        for filename in ['train.jsonl', 'valid.jsonl']:
            dataset[filename.split('.')[0]] = self._build_data(data_path + '/' + filename)
        return dataset

    @staticmethod
    def _build_data(data_path):
        data = {}
        with open(data_path, 'r') as f:
            for line in f:
                jline = json.loads(line)
                if 'label' in jline:
                    data[jline['premise'], jline['hypothesis']] = int(jline['label'] == 'entailment')
        return list(data.items())

In [ ]:
reader_terra = TERRaReader()
data_terra = reader_terra.read(data_path='/users/camila/course/TERRa')

In [ ]:
data_terra['train'][0]

(('Женщину доставили в больницу, за ее жизнь сейчас борются врачи.',
  'Женщину спасают врачи.'),
 1)

In [ ]:
chainer_config_terra = dict()
chainer_config_terra['in'] = ['text_a', 'text_b']
chainer_config_terra['in_y'] = ['y']
preproc_terra = {'class_name': "bert_preprocessor", 
            'vocab_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
           'do_lower_case': False,
           'max_seq_length': 64,
           'in': ['text_a', 'text_b'],
           'out': ['bert_features']}
classifier_terra = {'class_name': 'bert_classifier',
              'n_classes': 2,
              'one_hot_labels': False,
              'bert_config_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
              'pretrained_bert': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
              'save_path': "/users/camila/course/models/terra/paraph_terra",
              'load_path': "/users/camila/course/models/terra/terra",
              'keep_prob': 0.5,
              'optimizer': 'tf.train.AdamOptimizer',
              'learning_rate': 2e-5,
              'learning_rate_drop_patience': 3,
              'learning_rate_drop_div': 2,
              'in': ['bert_features'],
              'in_y': ['y'],
              'out': ['predictions']}
chainer_config_terra['pipe'] = [preproc_terra, classifier_terra]
chainer_config_terra['out'] = ['predictions']

In [ ]:
trainer_terra = NNTrainer(chainer_config=chainer_config_terra, batch_size=64,
                       train_metrics=['acc'], metrics=['acc'], validation_patience=7, 
                       val_every_n_batches=50, log_every_n_batches=50)

In [ ]:
iterator_terra = SiameseIterator(data=data_terra, seed=243, len_valid=500)

In [ ]:
trainer_terra.train(iterator_terra)

2021-04-22 00:52:52.936 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /users/camila/course/models/terra/terra]


INFO:tensorflow:Restoring parameters from /users/camila/course/models/terra/terra


2021-04-22 00:53:14.952 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best acc of 0.6743


{"valid": {"eval_examples_count": 307, "metrics": {"acc": 0.6743}, "time_spent": "0:00:20", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 7}}


2021-04-22 00:53:34.518 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 339: Stopped training


In [ ]:
chainer_config = dict()
chainer_config['in'] = ['text_a', 'text_b']
chainer_config['in_y'] = ['y']
preproc = {'class_name': "bert_preprocessor", 
            'vocab_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
           'do_lower_case': False,
           'max_seq_length': 128,
           'in': ['text_a', 'text_b'],
           'out': ['bert_features']}
classifier = {'class_name': 'bert_classifier',
              'n_classes': 2,
              'one_hot_labels': False,
              'bert_config_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
              'pretrained_bert': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
              'save_path': "/users/camila/course/models/danetqa/paraph_terra",
              'load_path': "/users/camila/course/models/terra/paraph_terra", #
              'keep_prob': 0.5,
              'optimizer': 'tf.train.AdamOptimizer',
              'learning_rate': 3e-5, #
              'learning_rate_decay': 'linear',
              'learning_rate_decay_epochs': 40, #
              'in': ['bert_features'],
              'in_y': ['y'],
              'out': ['predictions']}
chainer_config['pipe'] = [preproc, classifier]
chainer_config['out'] = ['predictions']

In [ ]:
trainer = NNTrainer(chainer_config=chainer_config, epochs=40, batch_size=32,
                       train_metrics=['acc'], metrics=['acc'], #
                       val_every_n_epochs=1, log_every_n_epochs=1,
                       evaluation_targets=['valid', 'train'], tensorboard_log_dir=MODELS_PATH + '/boolqa_rubert/logs')

In [ ]:
trainer.train(iterator)

2021-04-22 00:54:40.721 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /users/camila/course/models/terra/paraph_terra]


INFO:tensorflow:Restoring parameters from /users/camila/course/models/terra/paraph_terra


2021-04-22 00:56:41.351 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best acc of 0.5006


{"valid": {"eval_examples_count": 819, "metrics": {"acc": 0.5006}, "time_spent": "0:01:59", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 0.75}, "time_spent": "0:18:35", "epochs_done": 1, "batches_seen": 55, "train_examples_seen": 1749, "learning_rate": 3e-05, "loss": 0.7461211627179926}}


2021-04-22 01:15:05.37 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 207: Improved best acc of 0.5421
2021-04-22 01:15:05.39 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 209: Saving model
2021-04-22 01:15:05.49 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /users/camila/course/models/danetqa/paraph_terra]


...

2021-04-22 04:21:14.827 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.6679


{"valid": {"eval_examples_count": 819, "metrics": {"acc": 0.6606}, "time_spent": "3:26:32", "epochs_done": 11, "batches_seen": 605, "train_examples_seen": 19239, "impatience": 3, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 1.0}, "time_spent": "3:43:16", "epochs_done": 12, "batches_seen": 660, "train_examples_seen": 20988, "learning_rate": 2.175e-05, "loss": 0.006961333661721173}}


2021-04-22 04:39:44.803 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 207: Improved best acc of 0.6752
2021-04-22 04:39:44.806 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 209: Saving model
2021-04-22 04:39:44.824 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 75: [saving model to /users/camila/course/models/danetqa/paraph_terra]


{"valid": {"eval_examples_count": 819, "metrics": {"acc": 0.6752}, "time_spent": "3:45:02", "epochs_done": 12, "batches_seen": 660, "train_examples_seen": 20988, "impatience": 0, "patience_limit": 5}}


2021-04-22 04:41:23.240 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 339: Stopped training


In [ ]:
import json

In [ ]:
def read(data_path, language, *args, **kwargs):
    dataset = {}

    for filename in ['test.jsonl']:
        dataset[filename.split('.')[0]] = build_data(language, data_path + filename)

    return dataset

def build_data(ln, data_path):

    data = []
    with open('test.jsonl', 'r') as f:
        for line in f:
            jline = json.loads(line)
            if ln == 'ru':
                data.append((jline['question'], jline['passage']))

    return data

In [ ]:
%cd DaNetQA

[Errno 2] No such file or directory: 'DaNetQA'
/content/drive/My Drive/colab/DaNetQA


In [ ]:
data['test'] = read('', 'ru')['test']

In [ ]:
%cd ..

/content/drive/My Drive/colab


In [ ]:
len(data['test'])

805

In [ ]:
from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor

proc = BertPreprocessor(vocab_file=DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
           do_lower_case=False,
           max_seq_length=128)

In [ ]:
from deeppavlov.models.bert.bert_classifier import BertClassifierModel

clf = BertClassifierModel(n_classes=2,
              one_hot_labels=False,
              bert_config_file=DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
              pretrained_bert=DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
              save_path="content/drive/MyDrive/colab/muserc_danetqa",
              load_path="muserc/danetqa_muserc", #
              keep_prob=0.5,
              optimizer=tf.train.AdamOptimizer,
              learning_rate=3e-5,
              return_probas=True,)

2021-05-10 10:01:32.203 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/drive/My Drive/colab/muserc/danetqa_muserc' differs from save path '/content/drive/My Drive/colab/content/drive/MyDrive/colab/muserc_danetqa' in 'infer' mode for BertClassifierModel.








The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-05-10 10:01:55.857 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/muserc/danetqa_muserc]



INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/muserc/danetqa_muserc


In [ ]:
x_valid, y_valid = iterator.get_instances(data_type='valid')
x_valid_a = [obj[0] for obj in x_valid]
x_valid_b = [obj[1] for obj in x_valid]

In [ ]:
x_valid_a[0]

'Есть ли вода на марсе?'

In [ ]:
import numpy as np

In [ ]:
y_valid_pred = clf(proc(x_valid_a, x_valid_b))

In [ ]:
best_th_score = 0
best_th = 0.5
for th in (0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95):
    y_valid_pred1 = list(map(int, [obj[1] >= th for obj in y_valid_pred]))
    score = np.sum(np.array(y_valid) == y_valid_pred1) / len(y_valid)
    if score > best_th_score:
        best_th_score = score
        best_th = th
        print("New best th: {} with ACC = {}".format(best_th, best_th_score))

New best th: 0.5 with ACC = 0.6263736263736264
New best th: 0.55 with ACC = 0.6288156288156288
New best th: 0.6 with ACC = 0.6349206349206349
New best th: 0.65 with ACC = 0.6361416361416361
New best th: 0.7 with ACC = 0.6410256410256411
New best th: 0.75 with ACC = 0.6446886446886447
New best th: 0.8 with ACC = 0.6495726495726496
New best th: 0.9 with ACC = 0.6617826617826618
New best th: 0.95 with ACC = 0.6788766788766789


In [ ]:
%cd drive/MyDrive/colab

/content/drive/MyDrive/colab


In [ ]:
%cd MuSeRC

/content/drive/MyDrive/colab/MuSeRC


In [ ]:
import json

In [ ]:
def muserc(data):
    res = {}
    file = open(data, 'r')
    data = file.readlines()
    for passage in data:
        passage = json.loads(passage)
        text = passage["passage"]["text"]
        for query in passage["passage"]["questions"]:
            question = query["question"]
            for result in query["answers"]:
                answer = result["text"]
                label = result.get("label", 0)
                res[text, question + ' ' + answer] = label
    return list(res.items())

def muserc_read():
    dataset = {}

    for filename in ['train.jsonl', 'valid.jsonl', 'test.jsonl']:
        dataset[filename.split('.')[0]] = muserc(filename)

    return dataset

In [ ]:
data_m = muserc_read()

In [ ]:
data_m['valid'][0]

(('(1) Самый первый «остров» Архипелага возник в 1923 году на месте Соловецкого монастыря. (2) Затем появились ТОНы — тюрьмы особого назначения и этапы. (3) Люди попадали на Архипелаг разными способами: в вагон-заках, на баржах, пароходах и пешими этапами. (4) В тюрьмы арестованных доставляли в «воронках» — фургончиках чёрного цвета. (5) Роль портов Архипелага играли пересылки, временные лагеря, состоящие из палаток, землянок, бараков или участков земли под открытым небом. (6) На всех пересылках держать «политических» в узде помогали специально отобранные урки, или «социально близкие». (7) Солженицын побывал на пересылке Красная Пресня в 1945 году. (8) Эмигранты, крестьяне и «малые народы» перевозили красными эшелонами. (9) Чаще всего такие эшелоны останав\xadливались на пустом месте, посреди степи или тайги, и осуждённые сами строили лагерь. (10) Особо важные заключённые, в основном учёные, перевозились спецконвоем. (11) Так перевозили и Солженицына. (12) Он назвался ядерным физиком, 

In [ ]:
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

iterator_m = BasicClassificationDatasetIterator(data=data_m, seed=243)

In [ ]:
chainer_config = dict()
chainer_config['in'] = ['text_a', 'text_b']
chainer_config['in_y'] = ['y']
preproc = {'class_name': "bert_preprocessor", 
            'vocab_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt",
           'do_lower_case': False,
           'max_seq_length': 128,
           'in': ['text_a', 'text_b'],
           'out': ['bert_features']}
classifier = {'class_name': 'bert_classifier',
              'n_classes': 2,
              'one_hot_labels': False,
              'bert_config_file': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
              'pretrained_bert': DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
              'save_path': "muserc/muserc1",
              'load_path': "muserc/muserc1",
              'keep_prob': 0.5,
              'optimizer': 'tf.train.AdamOptimizer',
              'learning_rate': 3e-5, #
              'learning_rate_decay': 'linear',
              'learning_rate_decay_epochs': 5, #
              'in': ['bert_features'],
              'in_y': ['y'],
              'out': ['predictions']}
chainer_config['pipe'] = [preproc, classifier]
chainer_config['out'] = ['predictions']

In [ ]:
%cd ..

/content/drive/My Drive/colab


In [ ]:
from deeppavlov.core.trainers import NNTrainer

In [ ]:
trainer_m = NNTrainer(chainer_config=chainer_config, epochs=5, batch_size=32,
                       train_metrics=['acc'], metrics=['acc'], #
                       val_every_n_epochs=1, log_every_n_epochs=1,
                       val_every_n_batches=50, log_every_n_batches=50,
                       evaluation_targets=['valid', 'train'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


In [ ]:
len(data_m['train'])

11945

In [38]:
!pip install -r deeppavlov/requirements/bert_dp.txt

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-p9bdvq9u
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-p9bdvq9u
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=24eade9cb4ee9a13d21a98161e5ce77884a5ca8c5435ff7674dcecad9c4714ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-m2h5cqej/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [ ]:
trainer_m.train(iterator_m)











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-05-07 17:53:41.113 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/muserc/muserc1]



INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/muserc/muserc1


2021-05-07 18:14:56.958 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 199: Initial best acc of 0.736


{"valid": {"eval_examples_count": 2235, "metrics": {"acc": 0.736}, "time_spent": "0:21:10", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 0.8125}, "time_spent": "1:02:06", "epochs_done": 0, "batches_seen": 50, "train_examples_seen": 1600, "learning_rate": 3e-05, "loss": 0.37590896695852277}}


...

{"valid": {"eval_examples_count": 2235, "metrics": {"acc": 0.7217}, "time_spent": "3:32:13", "epochs_done": 0, "batches_seen": 150, "train_examples_seen": 4800, "impatience": 3, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 0.8125}, "time_spent": "4:13:11", "epochs_done": 0, "batches_seen": 200, "train_examples_seen": 6400, "learning_rate": 3e-05, "loss": 0.4918356388807297}}


2021-05-07 22:30:24.972 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.736


{"valid": {"eval_examples_count": 2235, "metrics": {"acc": 0.7123}, "time_spent": "4:36:38", "epochs_done": 0, "batches_seen": 200, "train_examples_seen": 6400, "impatience": 4, "patience_limit": 5}}
{"train": {"eval_examples_count": 32, "metrics": {"acc": 0.9375}, "time_spent": "5:17:43", "epochs_done": 0, "batches_seen": 250, "train_examples_seen": 8000, "learning_rate": 3e-05, "loss": 0.4750043910741806}}


2021-05-07 23:35:19.774 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 212: Did not improve on the acc of 0.736
2021-05-07 23:35:19.782 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 308: Ran out of patience


{"valid": {"eval_examples_count": 2235, "metrics": {"acc": 0.7239}, "time_spent": "5:41:32", "epochs_done": 0, "batches_seen": 250, "train_examples_seen": 8000, "impatience": 5, "patience_limit": 5}}


In [ ]:
x_test_a = [obj[0] for obj in data['test']]
x_test_b = [obj[1] for obj in data['test']]

In [ ]:
len(x_test_b)

805

In [ ]:
y_test = []
ths = []

In [ ]:
from deeppavlov.models.bert.bert_classifier import BertClassifierModel

[nltk_data] Downloading package punkt to /Users/camila/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/camila/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/camila/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/camila/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [ ]:
%cd ..

/content/drive/My Drive/colab


In [ ]:
y_test = []

In [ ]:
for model in ['DaNetQA/danetqa/danetqa', 'DaNetQA/danetqa/paraph_terra', 'DaNetQA/danetqa/terra', 'muserc/danetqa_muserc']:#, 'DaNetQA/danetqa_pp/danetqa_pp']:
    clf = BertClassifierModel(n_classes=2,
                  one_hot_labels=False,
                  bert_config_file=DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json",
                  pretrained_bert=DOWNLOADS_PATH + "/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt",
                  save_path= 'DaNetQA/exp/exp',
                  load_path= model,
                  keep_prob=0.5,
                  optimizer=tf.train.AdamOptimizer,
                  learning_rate=3e-5,
                  return_probas=True)
    '''y_valid_pred = clf(proc(x_valid_a, x_valid_b))
    best_th_score = 0
    best_th = 0.5
    print(model)
    for th in (0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95):
        y_valid_pred1 = list(map(int, [obj[1] >= th for obj in y_valid_pred]))
        score = np.sum(np.array(y_valid) == y_valid_pred1) / len(y_valid)
        if score > best_th_score:
            best_th_score = score
            best_th = th
            print("New best th: {} with ACC = {}".format(best_th, best_th_score))'''
    y_test_proba = clf(proc(x_test_a, x_test_b))
    #y_test_pred = list(map(int, [obj[1] >= best_th for obj in y_test_proba]))
    y_test.append(y_test_proba)

2021-05-10 13:13:06.278 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/drive/My Drive/colab/DaNetQA/danetqa/danetqa' differs from save path '/content/drive/My Drive/colab/DaNetQA/exp/exp' in 'infer' mode for BertClassifierModel.
2021-05-10 13:13:26.605 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/DaNetQA/danetqa/danetqa]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/DaNetQA/danetqa/danetqa


2021-05-10 13:20:56.837 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/drive/My Drive/colab/DaNetQA/danetqa/paraph_terra' differs from save path '/content/drive/My Drive/colab/DaNetQA/exp/exp' in 'infer' mode for BertClassifierModel.
2021-05-10 13:21:15.638 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/DaNetQA/danetqa/paraph_terra]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/DaNetQA/danetqa/paraph_terra


2021-05-10 13:28:56.374 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/drive/My Drive/colab/DaNetQA/danetqa/terra' differs from save path '/content/drive/My Drive/colab/DaNetQA/exp/exp' in 'infer' mode for BertClassifierModel.
2021-05-10 13:29:16.673 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/DaNetQA/danetqa/terra]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/DaNetQA/danetqa/terra


2021-05-10 13:36:45.293 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/drive/My Drive/colab/muserc/danetqa_muserc' differs from save path '/content/drive/My Drive/colab/DaNetQA/exp/exp' in 'infer' mode for BertClassifierModel.
2021-05-10 13:37:04.947 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/My Drive/colab/muserc/danetqa_muserc]


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/colab/muserc/danetqa_muserc


In [ ]:
y_test_pp = list(map(int, [obj[1] >= ths[4] for obj in y_test[0]]))
y_test4 = y_test[0]

In [ ]:
y_test.append(y_test4)

In [ ]:
file = open('res.txt', 'w')
for y in y_res:
    print(y, file=file)

In [ ]:
y_test_preds = []
for i in range(5):
    y_test_preds.append(list(map(int, [obj[1] >= ths[i] for obj in y_test[i]])))

In [ ]:
y_res = []
for i in range(len(y_test[0])):
    if y_test_preds[1][i] + y_test_preds[2][i] + y_test_preds[4][i] >= 2:
        y_res.append(1)
    else:
        y_res.append(0)

In [ ]:
y_valid_pred = clf1(proc(x_valid_a, x_valid_b))

In [ ]:
best_th_score = 0
best_th = 0.5
for th in (0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95):
    y_valid_pred1 = list(map(int, [obj[1] >= th for obj in y_valid_pred]))
    score = np.sum(np.array(y_valid) == y_valid_pred1) / len(y_valid)
    if score > best_th_score:
        best_th_score = score
        best_th = th
        print("New best th: {} with ACC = {}".format(best_th, best_th_score))

New best th: 0.5 with ACC = 0.5970695970695971
New best th: 0.8 with ACC = 0.5982905982905983
New best th: 0.85 with ACC = 0.6007326007326007
New best th: 0.9 with ACC = 0.6019536019536019
New best th: 0.95 with ACC = 0.6153846153846154


In [ ]:
iterator = BasicClassificationDatasetIterator(data=data, seed=243)

In [ ]:
x_test_a = [obj[0] for obj in data['test']]
x_test_b = [obj[1] for obj in data['test']]

In [ ]:
y_test_pred = clf(proc(x_test_a, x_test_b))

In [ ]:
y_test_proba = list(map(int, [obj[1] >= 0.9 for obj in y_test_pred]))

In [ ]:
file = open('DaNetQA5.txt', 'w')
for i in range(len(y_res)):
    print(y_res[i], file=file)

In [ ]:
y_res

[0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
